In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from dataset import make_dataset
from os.path import join


train_path = join('jeju_data', 'train_new.parquet')
test_path = join('jeju_data', 'test_new.parquet')

x_train, y_train, test = make_dataset(train_path, test_path)

시작시간: 2022-10-17 17:08:17.103751
Train dataset success !
Test dataset success !
끝난 시간:  0:01:06.058098


In [2]:
import pandas as pd

X = x_train.copy()
y = y_train.copy()
sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

Best trial {'learning_rate': 0.047360814959130804, 'bagging_temperature': 40.52044595178952, 'n_estimators': 3830, 'max_depth': 16, 'random_strength': 86, 'colsample_bylevel': 0.4129957917974832, 'l2_leaf_reg': 1.7654281892238895e-05, 'min_child_samples': 24, 'max_bin': 378, 'od_type': 'Iter'}


In [1]:
params = {'learning_rate': 0.047360814959130804, 'bagging_temperature': 40.52044595178952, 'n_estimators': 3830, 'max_depth': 16, 'random_strength': 86, 'colsample_bylevel': 0.4129957917974832, 'l2_leaf_reg': 1.7654281892238895e-05, 'min_child_samples': 24, 'max_bin': 378, 'od_type': 'Iter'}


from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

CAT_model= {}

for f in range(5):
      print(f'===================================={f+1}============================================')
      train_idx, val_idx = folds[f]
      
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]
      
      CAT = CatBoostRegressor(**params)
      CAT.fit(x_train, y_train)
      
      y_pred = CAT.predict(x_val)
      mae = mean_absolute_error(y_val, y_pred)
      print(f"{f + 1} Fold MAE = {mae}")
      CAT_model[f] = CAT
      print(f'================================================================================\n\n')
              


for fold in range(5):
    sample_submission['target'] += CAT_model[fold].predict(test)/5    




In [ ]:
sample_submission.to_csv("./submit_cat_kfold.csv", index=False)